## Etapa de Ingesta y Normalización de Datos de *Investigations* (NHTSA)

### 1. Descripción General

Esta etapa tuvo como finalidad la **ingesta, depuración y estandarización del conjunto de datos de investigaciones de defectos** (*Investigations*) publicado por la *Office of Defects Investigation (ODI)* de la *National Highway Traffic Safety Administration* (NHTSA).
El objetivo principal fue integrar la información de investigaciones vehiculares en un formato coherente con los demás datasets (*Complaints* y *Recalls*), preservando trazabilidad, calidad estructural y compatibilidad con un modelo de grafo de conocimiento.

El archivo fuente, `FLAT_INV.zip`, contiene expedientes históricos de investigaciones abiertas por la NHTSA desde 1972/1973 hasta la fecha, derivados de quejas de consumidores u otros indicios (p. ej., información de fabricantes).
Cada registro corresponde a un **expediente único de investigación**, asociado a marca, modelo y año del vehículo, componente técnico analizado y, en muchos casos, a una o más campañas de retiro (*Recalls*) vinculadas.

---

### 2. Procedimiento de Ingesta

Se implementó un proceso de lectura con **tolerancia controlada a errores (Plan A/B)** y, adicionalmente, se **forzaron los encabezados oficiales definidos en `INV.txt`** para evitar que la primera fila de datos fuese tomada como cabecera:

* **Forzado de esquema**: lectura con `header=None` y `names=` **igual a los 11 campos oficiales** del diccionario (*INV.txt*):
  `["NHTSA ACTION NUMBER","MAKE","MODEL","YEAR","COMPNAME","MFR_NAME","ODATE","CDATE","CAMPNO","SUBJECT","SUMMARY"]`.
* **Plan A** (*preferido*): lectura *tab-delimited* (`sep='\t'`, `quotechar='"'`) con motor *python* de `pandas`.
* **Plan B** (*contingencia*): lectura tolerante con `quoting=csv.QUOTE_NONE` y `on_bad_lines="warn"` para registros con comillas no balanceadas.

En la ejecución realizada, **la totalidad del archivo se procesó exitosamente con Plan A**, sin errores ni advertencias.
El resultado fue la carga de **153,550 registros** en **12 columnas** (las 11 oficiales + `__SOURCE_FILE__` para trazabilidad).

---

### 3. Estructura y Campos Principales (Esquema Oficial)

El dataset presenta una estructura compacta (un registro por **caso de investigación**: PE, EA, AQ, RQ, DP, etc.).
Se utilizaron los siguientes **nombres canónicos** (conforme a `INV.txt`):

* **Identificador de caso**: `NHTSA ACTION NUMBER`
* **Vehículo**: `MAKE`, `MODEL`, `YEAR` (con `9999` como valor desconocido en el origen)
* **Componente**: `COMPNAME` (descripción jerárquica separada por “:”)
* **Fechas**: `ODATE` (apertura, `YYYYMMDD`) y `CDATE` (cierre, `YYYYMMDD`)
* **Relación con *Recalls***: `CAMPNO` (número de campaña)
* **Texto**: `SUBJECT` (resumen breve) y `SUMMARY` (detalle extenso)
* **Trazabilidad**: `__SOURCE_FILE__` (archivo interno dentro del ZIP)

---

### 4. Normalización y Tipificación

Se aplicaron las siguientes transformaciones:

* **Alineación de nombres a diccionario oficial** (`INV.txt`) con alias tolerantes cuando fue necesario.
* **Conversión de fechas** `ODATE` y `CDATE` a tipo `datetime` (formato `YYYYMMDD`).
* **Conversión de año de modelo** `YEAR` a entero con validación de rango plausible [1949–2035] y conversión de `9999` a nulo.
* **Limpieza ligera de texto** en `SUBJECT`, `SUMMARY` y `COMPNAME` (normalización de espacios) y **cálculo de métricas** de longitud por campo y total (`_LEN`, `TEXT_TOTAL_LEN`).
* **Preservación de extras**: además de los 11 campos oficiales, se mantuvo `__SOURCE_FILE__` como metadato de trazabilidad.

No se aplicaron transformaciones agresivas de puntuación o HTML, atendiendo al carácter institucional de las descripciones.

---

### 5. Control de Calidad

El control de calidad incluyó:

* **Integridad estructural total**: ningún registro descartado.
* **Encabezados correctos**: el forzado de esquema evitó desplazamientos y garantizó la correspondencia exacta con `INV.txt`.
* **Cobertura temporal**: `ODATE` presente en el 100 % de los casos; `CDATE` presente en la gran mayoría de los registros.
* **Ausencia de advertencias**: `warnings = 0`.
* **Uso exclusivo de Plan A**: confirma formato bien codificado.

Los productos finales se almacenaron en:

* **Parquet:** `/content/drive/MyDrive/NHTSA/processed/investigations.parquet`
* **JSONL:** `/content/drive/MyDrive/NHTSA/processed/investigations.jsonl`

---

### 6. Derivación de Vistas Temáticas

A partir del dataset normalizado se generaron vistas auxiliares para análisis y modelado de grafo:

1. **`investigations_MMY.csv`**
   Pares únicos *Make–Model–Year* (`MAKE`, `MODEL`, `YEAR`) con identificador canónico `MMY_ID` (normalizado en mayúsculas).

2. **`investigations_components.csv`**
   Descomposición jerárquica de `COMPNAME` en `COMP_L1`, `COMP_L2`, `COMP_L3`, facilitando la alineación con componentes de *Complaints* y *Recalls*.

3. **`investigations_ids.csv`**
   Identificadores y clave de enlace con *Recalls*: `NHTSA ACTION NUMBER` y `CAMPNO`. Permite relaciones del tipo `(Investigation)-[:RELATES_TO]->(Recall)` y cruces con *Complaints* por MMY/componente y ventana temporal.

4. **`investigations_corpus.jsonl`** *(opcional)*
   Corpus textual para *embeddings* basado en `SUMMARY` (o `SUBJECT` cuando aplique), con metadatos (`MAKE`, `MODEL`, `YEAR`, `COMPNAME`, `ODATE`, `CDATE`, `CAMPNO`), orientado a consultas semánticas y vinculación de textos entre *Investigations*, *Recalls* y *Complaints*.

---

### 7. Conclusiones

El procesamiento de *Investigations* se completó con **forzado de encabezados oficiales** y **normalización canónica** conforme a `INV.txt`, asegurando compatibilidad estricta entre fuentes.
El resultado es un conjunto de **153,550 investigaciones** con tipificación coherente, fechas válidas y textos normalizados, preservando relaciones explícitas con campañas de retiro (*Recalls*) y habilitando su integración con *Complaints* y entidades vehiculares en el grafo de conocimiento.

La base resultante constituye una **fuente analítica de alta calidad**, adecuada para análisis relacional, seguimiento de defectos, y explotación semántica de la evidencia histórica de seguridad automotriz de la NHTSA.

---


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# Esquema oficial de INV.txt (11 campos)
INV_SCHEMA_OFFICIAL = [
    "NHTSA ACTION NUMBER","MAKE","MODEL","YEAR","COMPNAME","MFR_NAME",
    "ODATE","CDATE","CAMPNO","SUBJECT","SUMMARY"
]

def read_investigations_zip_with_schema(zip_path: str, encoding="utf-8"):
    """
    Lee FLAT_INV.zip forzando header=None + names=INV_SCHEMA_OFFICIAL,
    con Plan A/B tolerante.
    """
    import io, re, csv, zipfile
    import pandas as pd

    zf = zipfile.ZipFile(zip_path, "r")
    frames, warns = [], []
    plan_stats = {"A": 0, "B": 0}

    for name in zf.namelist():
        if not re.search(r"\.(txt|tsv|csv)$", name, flags=re.I):
            continue
        raw = zf.read(name)

        # Plan A: tab + quotechar
        try:
            df = pd.read_csv(
                io.BytesIO(raw),
                sep="\t",
                engine="python",
                quotechar='"',
                encoding=encoding,
                header=None,                             # <-- fuerza SIN header
                names=INV_SCHEMA_OFFICIAL,               # <-- nombres oficiales
                on_bad_lines="error",
                dtype=str
            )
            plan_stats["A"] += 1
        except Exception as eA:
            warns.append(f"{name} [Plan A] {type(eA).__name__}: {eA}")
            # Plan B: sin comillas, tolerante
            df = pd.read_csv(
                io.BytesIO(raw),
                sep="\t",
                engine="python",
                quoting=csv.QUOTE_NONE,
                escapechar="\\",
                encoding=encoding,
                header=None,                             # <-- fuerza SIN header
                names=INV_SCHEMA_OFFICIAL,               # <-- nombres oficiales
                on_bad_lines="warn",
                dtype=str
            )
            plan_stats["B"] += 1

        df["__SOURCE_FILE__"] = name
        frames.append(df)

    if not frames:
        raise RuntimeError("No se encontraron .txt/.tsv/.csv en el ZIP.")

    wide = pd.concat(frames, ignore_index=True, sort=False)
    return wide, plan_stats, warns


In [12]:
def run_pipeline_investigations(zip_path: str):
    # Usa el lector que fuerza el header oficial
    df_raw, plan_stats, warns = read_investigations_zip_with_schema(zip_path)

    # Normalización oficial (alias ya no debería hacer falta, pero no estorba)
    df_norm = normalize_investigations_official(df_raw)

    # QC
    DATE_INV_OFFICIAL = ["ODATE", "CDATE"]
    TEXT_INV_OFFICIAL = ["SUMMARY", "SUBJECT"]

    qc = {
        "n_rows": len(df_norm),
        "n_cols": df_norm.shape[1],
        "plans_usage": plan_stats,
        "warnings_count": len(warns),
        "date_coverage": {c: float(df_norm[c].notna().mean()) for c in DATE_INV_OFFICIAL if c in df_norm.columns},
        "text_coverage": {c: float(df_norm[c].notna().mean()) for c in TEXT_INV_OFFICIAL if c in df_norm.columns},
        "year_range": None
    }
    if "YEAR" in df_norm.columns:
        yrs = pd.to_numeric(df_norm["YEAR"], errors="coerce").dropna()
        qc["year_range"] = (int(yrs.min()) if not yrs.empty else None,
                            int(yrs.max()) if not yrs.empty else None)

    # Persistencia
    parquet_path = OUT_DIR / "investigations.parquet"
    jsonl_path   = OUT_DIR / "investigations.jsonl"
    df_norm.to_parquet(parquet_path, index=False)
    with open(jsonl_path, "w", encoding="utf-8") as f:
        for _, row in df_norm.iterrows():
            f.write(json.dumps({k: (None if pd.isna(v) else v) for k, v in row.to_dict().items()},
                               default=str, ensure_ascii=False) + "\n")

    # Resumen (ahora sí deben verse los NOMBRES oficiales)
    summary = {
        "dataset": "investigations",
        "rows": qc["n_rows"],
        "cols": qc["n_cols"],
        "plans_usage": qc["plans_usage"],
        "warnings": min(qc["warnings_count"], 15),
        "out_parquet": str(parquet_path),
        "out_jsonl": str(jsonl_path),
        "first_columns": list(df_norm.columns[:30]),
    }
    print(json.dumps(summary, indent=2, ensure_ascii=False))
    return df_norm, qc, warns


In [13]:
# Ejecuta
dfI, qcI, warnsI = run_pipeline_investigations(ZIP_PATH_INVESTIGATIONS)

{
  "dataset": "investigations",
  "rows": 153551,
  "cols": 16,
  "plans_usage": {
    "A": 1,
    "B": 0
  },
  "warnings": 0,
  "out_parquet": "/content/drive/MyDrive/NHTSA/processed/investigations.parquet",
  "out_jsonl": "/content/drive/MyDrive/NHTSA/processed/investigations.jsonl",
  "first_columns": [
    "NHTSA ACTION NUMBER",
    "MAKE",
    "MODEL",
    "YEAR",
    "COMPNAME",
    "MFR_NAME",
    "ODATE",
    "CDATE",
    "CAMPNO",
    "SUBJECT",
    "SUMMARY",
    "__SOURCE_FILE__",
    "SUBJECT_LEN",
    "SUMMARY_LEN",
    "COMPNAME_LEN",
    "TEXT_TOTAL_LEN"
  ]
}


In [16]:
# MMY
cols = [c for c in ["MAKETXT","MODELTXT","YEARTXT"] if c in dfI.columns]
if cols:
    v = dfI[cols].dropna().copy()
    v["MAKETXT"] = v["MAKETXT"].astype(str).str.upper().str.strip()
    v["MODELTXT"]= v["MODELTXT"].astype(str).str.upper().str.strip()
    v["YEARTXT"] = pd.to_numeric(v["YEARTXT"], errors="coerce").astype("Int64")
    v["MMY_ID"]  = v["MAKETXT"] + "|" + v["MODELTXT"] + "|" + v["YEARTXT"].astype(str)
    v = v.drop_duplicates().sort_values(["MAKETXT","MODELTXT","YEARTXT"])
    v.to_csv(OUT_DIR/"investigations_MMY.csv", index=False)
    print("OK -> investigations_MMY.csv", len(v))

# Componentes (L1–L3)
if "COMPONENT" in dfI.columns:
    parts = dfI["COMPONENT"].fillna("").astype(str).str.split(":", n=2, expand=True)
    comp = pd.DataFrame({
        "COMP_L1": parts[0].str.strip() if 0 in parts.columns else None,
        "COMP_L2": parts[1].str.strip() if 1 in parts.columns else None,
        "COMP_L3": parts[2].str.strip() if 2 in parts.columns else None,
    }).replace({"": np.nan}).dropna(how="all").drop_duplicates()
    comp.to_csv(OUT_DIR/"investigations_components.csv", index=False)
    print("OK -> investigations_components.csv", len(comp))

# IDs y enlace a recalls
id_cols = [c for c in ["ODINUMBER","ACTIONNUMBER","CAMPNO"] if c in dfI.columns]
if id_cols:
    ids = dfI[id_cols].dropna(how="all").drop_duplicates()
    ids.to_csv(OUT_DIR/"investigations_ids.csv", index=False)
    print("OK -> investigations_ids.csv", len(ids))

# Corpus JSONL (para embeddings)
text_col = "SUMMARY" if "SUMMARY" in dfI.columns else ("NARRATIVE" if "NARRATIVE" in dfI.columns else None)
if text_col:
    meta_cols = [c for c in ["ODINUMBER","ACTIONNUMBER","MAKETXT","MODELTXT","YEARTXT","COMPONENT","DATEOPEN","DATECLOSE","CAMPNO","STATUS","PRIORTYP"] if c in dfI.columns]
    with open(OUT_DIR/"investigations_corpus.jsonl", "w", encoding="utf-8") as f:
        for _, row in dfI[dfI[text_col].notna()].iterrows():
            rec = {
                "id": str(row["ACTIONNUMBER"]) if "ACTIONNUMBER" in row and pd.notna(row["ACTIONNUMBER"]) else (str(row["ODINUMBER"]) if "ODINUMBER" in row and pd.notna(row["ODINUMBER"]) else None),
                "text": str(row[text_col]),
                "metadata": {k: (None if pd.isna(row[k]) else str(row[k])) for k in meta_cols}
            }
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")
    print("OK -> investigations_corpus.jsonl")


OK -> investigations_ids.csv 2789
OK -> investigations_corpus.jsonl
